In [1]:
import datasets
import transformers
import pandas as pd
from datasets import Dataset
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
! pip install datasets transformers rouge-score nltk


In [3]:
!pip install transformers pyarabic
!git clone https://github.com/aub-mind/arabert 

fatal: destination path 'arabert' already exists and is not an empty directory.


In [4]:
!apt install git-lfs

Reading package lists... Done
Building dependency tree       
Reading state information... Done
git-lfs is already the newest version (2.9.2-1).
0 upgraded, 0 newly installed, 0 to remove and 93 not upgraded.


In [5]:
from huggingface_hub import notebook_login

notebook_login()

In [6]:
train_set = pd.read_excel("../input/arabic-summarization/Arabic/dataset_XL_sum_v1.0_train_ar.xlsx")

test_set = pd.read_excel("../input/arabic-summarization/Arabic/dataset_XL_sum_v1.0_test_ar.xlsx")
train_set['Summary'] = train_set['Summary'].replace(r'\n', '', regex = True)
train_set['Document'] = train_set['Document'].replace(r'\n', '', regex = True)

test_set['Summary'] = test_set['Summary'].replace(r'\n', '', regex = True)
test_set['Document'] = test_set['Document'].replace(r'\n', '', regex = True)
print(train_set.shape)
train_set = train_set.dropna()
print(train_set.shape)
print(test_set.shape)

(37246, 2)
(37245, 2)
(4687, 2)


In [7]:
train_set

,Document,Summary
0,بحلول عام 2050 ستحتاج مصر إلى 21 مليار متر مكع...,"""هل سيتم تغيير العبارة الشهيرة للمؤرخ اليوناني..."
1,وذكرت وكالة الأنباء المحلية (جي.إن.إس) أن جماع...,قالت الشرطة في القطاع الهندي من إقليم كشمير إن...
2,ووقع اختياره على واد عمقه 800 متر محاط بثماني ...,في عام 816، تجول راهب يدعى كوكاي، في المنحدرات...
3,مسلح حوثي في إب وقال المصدر إن المسلحين الحوثي...,"أكد مصدر في ""الحراك التهامي"" لأبناء محافظة الح..."
4,مكاتب العمل المفتوحة لا تسبب الضوضاء وتساعد في...,لا تعد مكاتب العمل المفتوحة مجرد مصدر للصخب وا...
...,...,...
37241,طاف متظاهرون بجثمان الناشط المصري ميدان التحري...,وصف التيار الشعبي المصري تقرير الطب الشرعي حول...
37242,وأضاف وزير الداخلية، جان جامبون، أن التهديد ال...,قال وزير الداخلية البلجيكي إن قوات الأمن تبحث ...
37243,يعد تركسون مدافعا بارزا عن قضايا البيئة وفي رس...,حث مستشار بيئي مقرب من البابا فرانسيس أتباع ال...
37244,مكان سقوط القذيفة في إسرائيل وأعلنت ألوية النا...,قال الجيش الإسرائيلي إن اثنين من جنوده أصيبا ب...


In [8]:
train_set.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 37245 entries, 0 to 37245
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Document  37245 non-null  object
 1   Summary   37245 non-null  object
dtypes: object(2)
memory usage: 872.9+ KB


In [9]:
test_set.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4687 entries, 0 to 4686
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Document  4687 non-null   object
 1   Summary   4687 non-null   object
dtypes: object(2)
memory usage: 73.4+ KB


In [10]:
train_ds = Dataset.from_pandas(train_set)
test_ds = Dataset.from_pandas(test_set)


In [11]:
train_ds

Dataset({
    features: ['Document', 'Summary', '__index_level_0__'],
    num_rows: 37245
})

In [12]:
train_dataset, validation_dataset= train_ds.train_test_split(test_size=0.15).values()
train_dataset = train_dataset.select(range(10))
validation_dataset = validation_dataset.select(range(10))

In [13]:
data_all_splits = datasets.DatasetDict({"train":train_dataset,"test":test_ds, "val":validation_dataset})

In [14]:
data_all_splits

DatasetDict({
    train: Dataset({
        features: ['Document', 'Summary', '__index_level_0__'],
        num_rows: 10
    })
    test: Dataset({
        features: ['Document', 'Summary'],
        num_rows: 4687
    })
    val: Dataset({
        features: ['Document', 'Summary', '__index_level_0__'],
        num_rows: 10
    })
})

In [15]:
!pip install rouge
from rouge import Rouge
rouge = Rouge()

In [16]:

from transformers import BertTokenizer, AutoModelForSeq2SeqLM, pipeline
from arabert.preprocess import ArabertPreprocessor
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

model_name="malmarjeh/mbert2mbert-arabic-text-summarization"

preprocessor = ArabertPreprocessor(model_name=model_name)

tokenizer = BertTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

The following encoder weights were not tied to the decoder ['bert/pooler']
The following encoder weights were not tied to the decoder ['bert/pooler']


In [17]:
WHITESPACE_HANDLER = lambda k: re.sub('\s+', ' ', re.sub('\n+', ' ', k.strip()))

In [18]:
max_input_length = 512
max_target_length = 128


def preprocess_function(examples):
    inputs = [doc for doc in examples["Document"]]
    #inputs=preprocessor.preprocess(inputs)
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            examples["Summary"], max_length=max_target_length, truncation=True
        )

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [19]:
tokenized_datasets = data_all_splits.map(preprocess_function, batched=True)

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/5 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [20]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['Document', 'Summary', '__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 10
    })
    test: Dataset({
        features: ['Document', 'Summary', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 4687
    })
    val: Dataset({
        features: ['Document', 'Summary', '__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 10
    })
})

In [21]:
batch_size =4
model_name = model_name.split("/")[-1]
args = Seq2SeqTrainingArguments(
    f"{model_name}-finetuned-xsum_arabic_abstractive_final_final_correted",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=2,
    predict_with_generate=True,
    fp16=True,
    push_to_hub=True,
    overwrite_output_dir=True
)

In [22]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)


In [23]:
from datasets import load_metric
metric = load_metric("rouge")

In [24]:
import nltk
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]
    
    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    # Extract a few results
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}
    
    # Add mean generated length
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)
    
    return {k: round(v, 4) for k, v in result.items()}

In [25]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['Document', 'Summary', '__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 10
    })
    test: Dataset({
        features: ['Document', 'Summary', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 4687
    })
    val: Dataset({
        features: ['Document', 'Summary', '__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 10
    })
})

In [27]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["val"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics)

Cloning https://huggingface.co/lilouuch/mbert2mbert-arabic-text-summarization-finetuned-xsum_arabic_abstractive_final_final_correted into local empty directory.
Using cuda_amp half precision backend


In [28]:
import os
import gc
import torch
torch.cuda.empty_cache()
gc.collect()
os.environ["WANDB_DISABLED"] = "true"
trainer.train()

The following columns in the training set don't have a corresponding argument in `EncoderDecoderModel.forward` and have been ignored: __index_level_0__, token_type_ids, Summary, Document. If __index_level_0__, token_type_ids, Summary, Document are not expected by `EncoderDecoderModel.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 10
  Num Epochs = 2
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 1
  Total optimization steps = 6
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,No log,4.788510,0.000000,0.000000,0.000000,0.000000,22.000000
2,No log,4.170062,0.000000,0.000000,0.000000,0.000000,27.600000


The following columns in the evaluation set don't have a corresponding argument in `EncoderDecoderModel.forward` and have been ignored: __index_level_0__, token_type_ids, Summary, Document. If __index_level_0__, token_type_ids, Summary, Document are not expected by `EncoderDecoderModel.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 10
  Batch size = 4
The following columns in the evaluation set don't have a corresponding argument in `EncoderDecoderModel.forward` and have been ignored: __index_level_0__, token_type_ids, Summary, Document. If __index_level_0__, token_type_ids, Summary, Document are not expected by `EncoderDecoderModel.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 10
  Batch size = 4


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=6, training_loss=4.433552424112956, metrics={'train_runtime': 16.8692, 'train_samples_per_second': 1.186, 'train_steps_per_second': 0.356, 'total_flos': 7048711065600.0, 'train_loss': 4.433552424112956, 'epoch': 2.0})

In [ ]:
trainer.push_to_hub()
